In [1]:
import pandas as pd

In [2]:
#  Load data
df = pd.read_csv("dirty_cafe_sales.csv") 

In [3]:

print(df.shape)
print(df.info())


(10000, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
None


In [4]:
df.head()


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [5]:
df.describe(include='all')

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1891141,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [6]:
#Standardize missing/inconsistent markers like unknown,error,empty strings
def clean_specials(col):
 df[col] = df[col].replace(["UNKNOWN", "ERROR", "", " "], np.nan)


In [7]:
# 4. Convert to correct data types
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["Price Per Unit"] = pd.to_numeric(df["Price Per Unit"], errors="coerce")
df["Total Spent"] = pd.to_numeric(df["Total Spent"], errors="coerce")
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


In [8]:
#Check missing values after standardization
print(df.isna().sum())


Transaction ID         0
Item                 333
Quantity             479
Price Per Unit       533
Total Spent          502
Payment Method      2579
Location            3265
Transaction Date     460
dtype: int64


In [9]:
# 6. Handle missing values
#  - If Total Spent is missing but Price Per Unit and Quantity are available, compute: Price * Quantity
mask = df['Total Spent'].isna() & df['Price Per Unit'].notna() & df['Quantity'].notna()
df.loc[mask, 'Total Spent'] = df.loc[mask, 'Price Per Unit'] * df.loc[mask, 'Quantity']

# Similarly fill Price Per Unit or Quantity if missing and other two present.

# For categorical (Item, Payment Method, Location), fill missing with "Unknown" or mode
df['Payment Method'] = df['Payment Method'].fillna("Unknown")
df['Location'] = df['Location'].fillna("Unknown")
df['Item'] = df['Item'].fillna("Unknown")

# For date, maybe forward/backward fill or drop if many missing
df = df.sort_values('Transaction Date')
df['Transaction Date'] = df['Transaction Date'].fillna(method='ffill')  


In [10]:
# 7. Remove duplicates if any
df = df.drop_duplicates()


In [11]:
#Filter out rows that are still too bad (e.g. missing everything or invalid negative values etc.)
# remove rows where Quantity <= 0 or Price Per Unit <=0 etc.
df = df[(df['Quantity'] > 0) & (df['Price Per Unit'] > 0)]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9006 entries, 8015 to 9988
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    9006 non-null   object        
 1   Item              9006 non-null   object        
 2   Quantity          9006 non-null   float64       
 3   Price Per Unit    9006 non-null   float64       
 4   Total Spent       9006 non-null   float64       
 5   Payment Method    9006 non-null   object        
 6   Location          9006 non-null   object        
 7   Transaction Date  9006 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 633.2+ KB


In [13]:
print(df.isna().sum())


Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


In [14]:
# Save cleaned DataFrame to CSV
df.to_csv("cleaned_cafe_sales.csv", index=False)